# Chatbots

Following: https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [1]:
from langchain_community.chat_models import ChatOllama

In [2]:
model = ChatOllama(model="llama3")

## General Chat

In [3]:
from langchain_core.messages import HumanMessage

In [4]:
model.invoke([HumanMessage(content="Hello, I'm Oliver")])

AIMessage(content="Nice to meet you, Oliver! What brings you here today? Do you have a particular topic in mind that you'd like to chat about, or are you just looking for some friendly conversation? I'm all ears!", response_metadata={'model': 'llama3', 'created_at': '2024-06-06T19:15:33.912142Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 10895161300, 'load_duration': 10037605700, 'prompt_eval_count': 14, 'prompt_eval_duration': 65887000, 'eval_count': 45, 'eval_duration': 788828000}, id='run-e849f0f8-9f5c-434a-a74f-98a338497710-0')

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content="I know this one, Bob! Your name is... (drumroll) ...Bob! That's right, we're having a conversation and your name is indeed Bob!", response_metadata={'model': 'llama3', 'created_at': '2024-06-06T19:15:36.9906086Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1002281800, 'load_duration': 8526600, 'prompt_eval_count': 35, 'prompt_eval_duration': 299424000, 'eval_count': 36, 'eval_duration': 665168000}, id='run-c102f1cf-c1b2-43a2-8d94-3eb67f3a4427-0')

## Message history

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.config import RunnableConfig

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [7]:
config = RunnableConfig(configurable={"session_id": "1337"})

In [8]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)

In [9]:
response.content

"Nice to meet you, Oliver! What brings you here today? Do you have any questions or topics you'd like to discuss?"

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")], config=config)

In [11]:
response.content

'Your name is Oliver!'

# Prompt templates

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [13]:
response = chain.invoke({"messages": [HumanMessage(content="Hello, I'm Oliver")]})
response.content

"Nice to meet you, Oliver! It's great that you're reaching out and saying hello. How can I assist or chat with you today? Do you have something on your mind, or would you like to talk about a particular topic? I'm all ears (or rather, all text)!"

And now with history:

In [14]:
with_history = RunnableWithMessageHistory(chain, get_session_history)

In [15]:
config = RunnableConfig(configurable={"session_id": "1338"})

In [16]:
response = with_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)
response.content

"Nice to meet you, Oliver! What brings you here today? Do you have any specific questions or topics you'd like to discuss? I'm all ears (or in this case, all text)!"

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="Write a haiku that greets me by my name")],
    config=config,
)

print(response.content)

Oliver's gentle
Sunshine warms the morning dew
Welcome, dear friend


## More template variables

In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [25]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'¡Hola Bob! ¡Bienvenido! ¿En qué puedo ayudarte hoy? (Hello Bob! Welcome! How can I help you today?)'

In [26]:
with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [27]:
config = {"configurable": {"session_id": "abc11"}}

In [30]:
response = with_history.invoke(
    input={"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

'¡Hola Todd! ¡Bienvenido! (Hello Todd! Welcome!) How can I help you today?'

## Restrict message size

In [31]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [32]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [33]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't think we've established your name yet! I'm here to help with any questions or topics you'd like to discuss. What's on your mind?"

In [34]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what flavor of ice cream do i like?")],
        "language": "English",
    }
)
response.content

'vanilla!'

# Streaming

In [36]:
for chunk in with_history.stream(
    {
        "messages": [HumanMessage(content="hi! I'm bob. Tell me a long joke")],
        "language": "English",
    },
    config=config,
):
    print(chunk.content, end=" ")

Hey  Bob !

 Here 's  one  that 's  a  bit  longer  than  usual :

 A  man  walked  into  a  bar  and  ordered  a  beer .  As  he  was  s ipping  his  drink ,  he  heard  a  voice  say ,  " Nice  tie !"  He  looked  around ,  but  there  was  nobody  nearby  who  could  have  said  it .

 A  few  minutes  later ,  he  heard  the  same  voice  say ,  " Beautiful  shirt !  You  look  great  today !"  Again ,  he  looked  around ,  but  he  couldn 't  find  anyone  who  might  have  spoken .

 He  asked  the  bartender  if  he  had  noticed  anyone  unusual  or  strange ,  and  the  bartender  replied ,  " Oh ,  that 's  just  the  peanuts .  They 're  complimentary ."

 How  was  that ?  Did  I  make  you  LOL ?  